VVIMP before anything, make sure you can connection is on: <br>
sudo ip link set can0 up type can bitrate 250000  <br>

check using this: <br>
ip link show can0  <br>

to deactivate: <br>
sudo ip link set can0 down


In [6]:
import can
print(can.VALID_INTERFACES)

frozenset({'serial', 'kvaser', 'vector', 'cantact', 'neousys', 'virtual', 'ixxat', 'etas', 'nican', 'socketcan', 'robotell', 'neovi', 'seeedstudio', 'usb2can', 'slcan', 'pcan', 'canalystii', 'socketcand', 'gs_usb', 'systec', 'nixnet', 'udp_multicast', 'iscan'})


In [17]:
bus = can.interface.Bus(channel='can0', interface='socketcan')
print("Connected to:", bus.channel_info)

SocketcanBus was not properly shut down


Connected to: socketcan channel 'can0'


In [203]:
# close connection
try:
    bus.shutdown()
except:
    pass


In [ ]:
# not using gs_usb -> using socketcan instead
import can
bus = can.Bus(interface='gs_usb', channel=0x606f, index=0, bitrate=250000)

CanInterfaceNotImplementedError: Cannot import module can.interfaces.gs_usb for CAN interface 'gs_usb': No module named 'usb'

In [18]:
# heartbeat check on Odrive

import can, time

print("Starting CAN interface...")

# Try by index first (most common)
try:
    bus = can.Bus(interface='socketcan', channel='can0', bitrate=250000)
except Exception as e:
    print("Index open failed, trying product ID form:", e)
    # ODrive USB-CAN product ID
    bus = can.Bus(interface='socketcan', channel='can0', index=0, bitrate=250000)

print("CAN opened:", bus)

# Listen a couple seconds for ODrive heartbeat frames
t0 = time.time()
while time.time() - t0 < 3.0:
    msg = bus.recv(0.5)
    if msg:
        print(msg)

SocketcanBus was not properly shut down


Starting CAN interface...
CAN opened: socketcan channel 'can0'
Timestamp: 1762463600.059999    ID:      020    S Rx E              DL:  8    00 00 00 00 00 00 02 f9     Channel: can0
Timestamp: 1762463600.060401    ID:      020    S Rx E              DL:  8    00 00 00 00 00 00 02 f9     Channel: can0
Timestamp: 1762463600.060840    ID:      020    S Rx E              DL:  8    00 00 00 00 00 00 02 f9     Channel: can0
Timestamp: 1762463600.061240    ID:      020    S Rx E              DL:  8    00 00 00 00 00 00 02 f9     Channel: can0
Timestamp: 1762463600.061594    ID:      020    S Rx E              DL:  8    00 00 00 00 00 00 02 f9     Channel: can0
Timestamp: 1762463600.062040    ID:      020    S Rx E              DL:  8    00 00 00 00 00 00 02 f9     Channel: can0
Timestamp: 1762463600.062465    ID:      020    S Rx E              DL:  8    00 00 00 00 00 00 02 f9     Channel: can0
Timestamp: 1762463600.062802    ID:      020    S Rx E              DL:  8    00 00 00 00 00 00 0

In [28]:
import can, struct, time

# setup the CAN interface 
bus = can.interface.Bus("can0", interface="socketcan")

def set_axis_state(node_id: int, state_id: int):
    """Set the axis state (1 = IDLE, 8 = CLOSED_LOOP_CONTROL, etc.)"""
    arb_id = (node_id << 5) | 0x07
    msg = can.Message(
        arbitration_id=arb_id,
        data=struct.pack('<I', state_id),
        is_extended_id=False
    )
    bus.send(msg)

def set_position(node_id: int, pos: float, vel_ff: int = 0, torque_ff: int = 0):
    """Command position in turns with optional feed-forwards."""
    arb_id = (node_id << 5) | 0x0C
    msg = can.Message(
        arbitration_id=arb_id,
        data=struct.pack('<fhh', pos, vel_ff, torque_ff),
        is_extended_id=False
    )
    bus.send(msg)

def set_velocity(node_id: int, vel: float):
    """Command velocity in turns/s."""
    arb_id = (node_id << 5) | 0x0D
    msg = can.Message(
        arbitration_id=arb_id,
        data=struct.pack('<f', vel),
        is_extended_id=False
    )
    bus.send(msg)

def read_feedback(node_id: int, timeout: float = 0.1):
    """Read encoder estimates (position, velocity) for a specific node."""
    expected_id = (node_id << 5) | 0x09
    msg = bus.recv(timeout)
    if msg and msg.arbitration_id == expected_id:
        return struct.unpack('<ff', bytes(msg.data))
    return None, None


def get_axis_state(node_id):
    arb_id = (node_id << 5) | 0x01  # heartbeat/status frame
    while True:
        msg = bus.recv(timeout=0.5)
        if msg and msg.arbitration_id == arb_id:
            error, state, result, traj_done = struct.unpack('<IBBB', bytes(msg.data[:7]))
            print(f"State: {state}, Error: {error}")


            import can

def clear_errors_and_enable(node_id: int, reenable_closed_loop: bool = True):
    """
    Clears all errors on the specified ODrive node via CAN,
    and optionally re-enters CLOSED_LOOP_CONTROL.
    """
    try:
        # ---- Clear Errors ----
        msg = can.Message(
            arbitration_id=(node_id << 5) | 0x18,  # 0x18 = Clear_Errors
            data=b'',
            is_extended_id=False
        )
        bus.send(msg)
        print(f"✅ Sent Clear_Errors to node {node_id}")
        time.sleep(0.1)

        # ---- Optionally re-enable closed loop ----
        if reenable_closed_loop:
            msg = can.Message(
                arbitration_id=(node_id << 5) | 0x07,  # 0x07 = Set_Axis_State
                data=struct.pack('<I', 8),             # 8 = CLOSED_LOOP_CONTROL
                is_extended_id=False
            )
            bus.send(msg)
            print(f"✅ Node {node_id} re-entered CLOSED_LOOP_CONTROL")

    except can.CanError as e:
        print(f"⚠️ CAN send failed for node {node_id}: {e}")
    except Exception as e:
        print(f"⚠️ Unexpected error for node {node_id}: {e}")



# # Clear only
# clear_errors_and_enable(0, reenable_closed_loop=False)

# Clear and re-enable closed loop
# clear_errors_and_enable(1)

set_axis_state(2, 1)
for i in range(10):
    fb = read_feedback(2)
    print(fb)
    time.sleep(0.2)  # wait 200 ms between reads

SocketcanBus was not properly shut down


(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(0.00027757883071899414, 0.017523175105452538)
(None, None)
(None, None)


In [ ]:
# clear_errors_and_enable(1)

# enable both ODrives
# set_axis_state(0, 8)
set_axis_state(1, 8)

# get_axis_state(0)
# get_axis_state(1)

# move them
# set_position(0, -2.5)
# set_position(1, 1.5)

# get_axis_state(0)
# get_axis_state(1)

# change speeds
# set_velocity(0, 2.0)
# set_velocity(1, -1.0)


# time.sleep(1.0)  
# stop
# set_axis_state(0, 1)
# set_axis_state(1, 1)

# read feedback
import time
def read_all_feedback(active_nodes, duration=1.0):
    """Listen for feedback frames from all ODrives for a set time."""
    feedback = {nid: (None, None) for nid in active_nodes}
    start = time.time()

    while time.time() - start < duration:
        msg = bus.recv(timeout=0.05)
        if not msg:
            continue
        cmd_id = msg.arbitration_id & 0x1F
        node_id = msg.arbitration_id >> 5

        if cmd_id == 0x09 and node_id in active_nodes:
            pos, vel = struct.unpack('<ff', bytes(msg.data))
            feedback[node_id] = (pos, vel)

    return feedback


# usage
nodes = [1]
data = read_all_feedback(nodes, duration=0.5)
for nid, (pos, vel) in data.items():
    print(f"Node {nid}: pos={pos:.3f}, vel={vel:.3f}")





TypeError: unsupported format string passed to NoneType.__format__

In [185]:
# setting to idle state

import struct, can, time

node_id = 0  #  whichever ODrive you want
# 0 is base
# 1 is other one
bus = can.interface.Bus("can0", interface="socketcan")

# send Set_Axis_State(IDLE) 
msg = can.Message(
    arbitration_id=(node_id << 5) | 0x07,
    data=struct.pack('<I', 1),  # 1 = IDLE
    is_extended_id=False
)
bus.send(msg)
print("✅ Axis set to IDLE")


SocketcanBus was not properly shut down


✅ Axis set to IDLE


In [ ]:
## for clearing errors 

import can
import struct
import time

node_id = 1  # change to your ODrive’s node ID
bus = can.interface.Bus(channel='can0', interface='socketcan')

# clear all errors
msg = can.Message(
    arbitration_id=(node_id << 5 | 0x18),  # 0x18 = Clear_Errors
    data=b'',  # no payload
    is_extended_id=False
)
bus.send(msg)
print("✅ Sent Clear_Errors")

# give it a moment
time.sleep(0.1)

# optionally, re-enable closed loop
msg = can.Message(
    arbitration_id=(node_id << 5 | 0x07),  # 0x07 = Set_Axis_State
    data=struct.pack('<I', 8),  # 8 = CLOSED_LOOP_CONTROL
    is_extended_id=False
)
bus.send(msg)
print("✅ Re-entered CLOSED_LOOP_CONTROL")

SocketcanBus was not properly shut down


✅ Sent Clear_Errors
✅ Re-entered CLOSED_LOOP_CONTROL


In [182]:
while True:
    msg = bus.recv(timeout=0.5)
    if msg and msg.arbitration_id == (node_id << 5 | 0x01):
        error, state, result, traj_done = struct.unpack('<IBBB', bytes(msg.data[:7]))
        print(f"State: {state}, Error: {error}")

State: 1, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0
State: 8, Error: 0


KeyboardInterrupt: 

In [ ]:
# check if python can detect any can 
# this will not show socketcan necessarily. do not use this as a test. use the cell below that opens can0 using socketcan
# DO NOT USE THIS - Doesnt do anyhting
import can
print(can.detect_available_configs())



In [49]:
# trying to config via can
import can
bus = can.interface.Bus(channel='can0', interface='socketcan')
print("Connected to:", bus.channel_info)
"""

python3 ODrive-CAN/can_restore_config.py \
  --channel can0 \
  --node-id 1 \
  --endpoints-json can_control/flat_endpoints.json \
  --config can_control/config.json \
  --save-config

"""

SocketcanBus was not properly shut down


Connected to: socketcan channel 'can0'


'\n\npython3 ODrive-CAN/can_restore_config.py   --channel can0   --node-id 1   --endpoints-json can_control/flat_endpoints.json   --config can_control/config.json   --save-config\n\n'

In [64]:
# save and reboot
import can

bus = can.interface.Bus(channel='can0', bustype='socketcan')

node_id = 1
base = (0x07 << 5) | node_id

save_id = base + 0x0C
reboot_id = base + 0x0D

# save
bus.send(can.Message(arbitration_id=save_id, is_extended_id=False, data=[]))
print(f"Sent SAVE_CONFIG to 0x{save_id:X}")

# reboot
bus.send(can.Message(arbitration_id=reboot_id, is_extended_id=False, data=[]))
print(f"Sent REBOOT to 0x{reboot_id:X}")

/tmp/ipykernel_98331/2743874559.py:4: DeprecationWarning: The 'bustype' argument is deprecated since python-can v4.2.0, and scheduled for removal in python-can v5.0.0. Use 'interface' instead.
  bus = can.interface.Bus(channel='can0', bustype='socketcan')
SocketcanBus was not properly shut down


Sent SAVE_CONFIG to 0xED
Sent REBOOT to 0xEE
